Andrew Nakamoto and Edward Yeung \\
CSE 447: NLP \\
02/10/2024

This project is a very heavily modified version of https://huggingface.co/docs/transformers/en/tasks/multiple_choice

# Setup and Data

### Environment Setup

In [1]:
!pip install accelerate transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00


In [2]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.8 MB/s eta 0:00:00


### Load Data

Load the datasets from mounted drive. For others using this notebook, you'll need to replace the filepaths specified here with the paths to the correct files from https://leaderboard.allenai.org/open_book_qa/submissions/get-started

In [3]:
from datasets import load_dataset
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

path_to_train = "/content/drive/MyDrive/NLP_FINAL_PROJECT/OpenBookQA-V1-Sep2018/Data/Main/train.jsonl"
path_to_dev = "/content/drive/MyDrive/NLP_FINAL_PROJECT/OpenBookQA-V1-Sep2018/Data/Main/dev.jsonl"
path_to_test = "/content/drive/MyDrive/NLP_FINAL_PROJECT/OpenBookQA-V1-Sep2018/Data/Main/test.jsonl"
data_files = {"train": path_to_train, "validation": path_to_dev, "test": path_to_test}

data = load_dataset("json", data_files=data_files)

Mounted at /content/drive/


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
# test the output
data["test"][0]

{'id': '8-343',
 'question': {'stem': 'A person wants to start saving money so that they can afford a nice vacation at the end of the year. After looking over their budget and expenses, they decide the best way to save money is to',
  'choices': [{'text': 'make more phone calls', 'label': 'A'},
   {'text': 'quit eating lunch out', 'label': 'B'},
   {'text': 'buy less with monopoly money', 'label': 'C'},
   {'text': 'have lunch with friends', 'label': 'D'}]},
 'answerKey': 'B'}

### Setup Sentence-BERT

In [5]:
# read in the crowdsourced facts
path_to_facts = "/content/drive/MyDrive/NLP_FINAL_PROJECT/OpenBookQA-V1-Sep2018/Data/Additional/crowdsourced-facts.txt"

with open(path_to_facts, 'r') as f:
    text = f.read()

facts = text.splitlines()[1:] # drop the empty line at the top of the file

In [6]:
# https://arxiv.org/abs/1908.10084
# https://www.sbert.net/
# https://huggingface.co/sentence-transformers

from sentence_transformers import SentenceTransformer, util
retriever_model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# facts are encoded by calling model.encode()
fact_embeddings = retriever_model.encode(facts, convert_to_tensor=False)

# Train model

Configure the model and the tokenizer

In [8]:
from transformers import AutoTokenizer
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model_name = "microsoft/deberta-v3-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
mcq_model = AutoModelForMultipleChoice.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Tokenize the data

In [ ]:
import torch
answer_key_to_numeric = {'A': 0, 'B': 1, 'C': 2, 'D': 3}

def preprocess_function(examples):
    first_sentences = [[question['stem']] * 4 for question in examples['question']]
    second_sentences = [
        [f"{item['text']}" for item in question['choices']] for question in examples['question']
    ]

    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    # setup the output from the tokenizer into groupings of 4
    d = {k: [v[i : i + 4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}
    # add numeric labels to each of the items - needed for model
    d['label'] = [answer_key_to_numeric[c] for c in examples['answerKey']]
    return d

In [22]:
import torch
answer_key_to_numeric = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
k = 10

def preprocess_function(examples):
    first_sentences = [[question['stem']] * 4 for question in examples['question']]
    second_sentences = [
        [f"{item['text']}" for item in question['choices']] for question in examples['question']
    ]

    stem_embeddings = retriever_model.encode(first_sentences, convert_to_tensor=False)
    cosine_scores = util.cos_sim(stem_embeddings, fact_embeddings)
    with torch.no_grad():
      bestk = torch.topk(cosine_scores, k=k, dim=-1).indices
    bestk_facts = [[". ".join(f"{facts[i]}" for i in row)] * 4 for row in bestk]
    bestk_facts = sum(bestk_facts, [])

    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    first_sentences = [clues + ". " + stem for clues, stem in zip(bestk_facts, first_sentences)]

    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    # setup the output from the tokenizer into groupings of 4
    d = {k: [v[i : i + 4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}
    # add numeric labels to each of the items - needed for model
    d['label'] = [answer_key_to_numeric[c] for c in examples['answerKey']]
    return d

In [23]:
tokenized_data = data.map(preprocess_function, batched=True)

Map:   0%|          | 0/4957 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [24]:
# test the output
tokenized_data["test"][0]["input_ids"]

[[1,
  12935,
  666,
  3412,
  260,
  57408,
  1731,
  3713,
  6829,
  262,
  906,
  265,
  2150,
  994,
  260,
  927,
  478,
  3747,
  479,
  274,
  295,
  1662,
  2150,
  292,
  15111,
  1136,
  260,
  997,
  266,
  540,
  13242,
  8620,
  529,
  261,
  266,
  1008,
  2943,
  260,
  34267,
  1377,
  269,
  1241,
  283,
  1377,
  638,
  17785,
  260,
  336,
  3059,
  71480,
  335,
  262,
  437,
  1095,
  269,
  1754,
  260,
  57408,
  2150,
  17624,
  51415,
  262,
  2150,
  260,
  7458,
  28235,
  2150,
  260,
  4665,
  529,
  264,
  365,
  278,
  437,
  260,
  21100,
  269,
  311,
  265,
  262,
  1671,
  857,
  634,
  641,
  275,
  645,
  261,
  529,
  261,
  263,
  31196,
  260,
  336,
  604,
  1654,
  264,
  564,
  3412,
  580,
  324,
  272,
  306,
  295,
  4096,
  266,
  1085,
  3673,
  288,
  262,
  513,
  265,
  262,
  395,
  260,
  643,
  562,
  360,
  308,
  1753,
  263,
  3670,
  261,
  306,
  2116,
  262,
  410,
  384,
  264,
  1213,
  580,
  269,
  264,
  2,
  365,
  310,


### Trainer object setup

Adapt the DataCollatorWithPadding to create a batch of examples. It's more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [25]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

In [26]:
@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

Setup evaluation

In [27]:
import evaluate

accuracy = evaluate.load("accuracy")

import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

Set up the trainer. Requires HuggingFace write permission, so you need to set up an access token.

In [16]:
from huggingface_hub import notebook_login

notebook_login()

In [29]:
training_args = TrainingArguments(
    output_dir=f"openbookqa_{model_name}_fact_retrieval_k_10",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    weight_decay=0.01,
    push_to_hub=True,
    optim="adamw_torch",
    logging_strategy="epoch"
)

trainer = Trainer(
    model=mcq_model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

### Evaluate the model

In [33]:
trainer.evaluate()

KeyboardInterrupt: 

### Train the model

In [30]:
# don't forget to potentially reinitialize the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.309300,0.504227,0.824000
2,0.126400,0.975619,0.816000
3,0.061100,1.033291,0.836000


Epoch,Training Loss,Validation Loss,Accuracy
1,0.309300,0.504227,0.824000
2,0.126400,0.975619,0.816000
3,0.061100,1.033291,0.836000
4,0.017400,1.176568,0.850000
5,0.002900,1.175136,0.848000


TrainOutput(global_step=3100, training_loss=0.10340834829115099, metrics={'train_runtime': 2237.8484, 'train_samples_per_second': 11.075, 'train_steps_per_second': 1.385, 'total_flos': 2.2636310551958304e+16, 'train_loss': 0.10340834829115099, 'epoch': 5.0})

Save the model

In [32]:
path_to_save = f"/content/drive/MyDrive/NLP_FINAL_PROJECT/models/{trainer.args.output_dir}"
trainer.save_model(path_to_save)

In [21]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/asn1814/deberta-v3-large_fact_retrieval_k_3/commit/3407524040acece405a60f6416cc920190bbcdca', commit_message='End of training', commit_description='', oid='3407524040acece405a60f6416cc920190bbcdca', pr_url=None, pr_revision=None, pr_num=None)